In [5]:
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

In [8]:
!unzip 'catdog.zip'

Archive:  catdog.zip
   creating: train/
   creating: train/Cat/
  inflating: train/Cat/0.jpg         
  inflating: train/Cat/1.jpg         
  inflating: train/Cat/2.jpg         
  inflating: train/Cat/cat.2405.jpg  
  inflating: train/Cat/cat.2406.jpg  
  inflating: train/Cat/cat.2436.jpg  
  inflating: train/Cat/cat.2437.jpg  
  inflating: train/Cat/cat.2438.jpg  
  inflating: train/Cat/cat.2439.jpg  
  inflating: train/Cat/cat.2440.jpg  
  inflating: train/Cat/cat.2441.jpg  
  inflating: train/Cat/cat.2442.jpg  
  inflating: train/Cat/cat.2443.jpg  
  inflating: train/Cat/cat.2444.jpg  
  inflating: train/Cat/cat.2445.jpg  
  inflating: train/Cat/cat.2446.jpg  
  inflating: train/Cat/cat.2447.jpg  
  inflating: train/Cat/cat.2448.jpg  
  inflating: train/Cat/cat.2449.jpg  
  inflating: train/Cat/cat.2450.jpg  
  inflating: train/Cat/cat.2451.jpg  
  inflating: train/Cat/cat.2452.jpg  
  inflating: train/Cat/cat.2453.jpg  
  inflating: train/Cat/cat.2454.jpg  
  inflating: train/Cat/

In [9]:
# Set the path to your training and validation data.
train_data_dir='/content/train'
validation_dir='/content/validation'

# Num of train and test sample.
num_train_samples= 337
num_validation_samples=59

In [10]:
# set the number of epoch and batch size.
epoch=5
batch_size=16

In [11]:
# Load the inception model with out top layers
base_model= InceptionV3(weights='imagenet',
                        include_top=False,
                        input_shape=(224,224,3),
                        )

87910968/87910968 [==============================] - 0s 0us/step


In [13]:
## Freeze the base model layers:
for layers in base_model.layers:
  # print(layers)
  layers.trainable=False

In [14]:
## Create a new model
model=Sequential()

## Add the base model as a layers
model.add(base_model)

## Add custom layers on top of the base model
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 flatten (Flatten)           (None, 51200)             0         
                                                                 
 dense (Dense)               (None, 256)               13107456  
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 34910497 (133.17 MB)
Trainable params: 13107713 (50.00 MB)
Non-trainable params: 21802784 (83.17 MB)
_________________________________________________________________


In [16]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [17]:
## Preprocess the training and validation data
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)
validation_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)

In [21]:
train_generator= train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary'
)
validation_generator=validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224,224),
    batch_size=batch_size,
    class_mode='binary'

)

Found 337 images belonging to 2 classes.
Found 59 images belonging to 2 classes.


In [23]:
model.fit(train_generator,
          steps_per_epoch= num_train_samples // batch_size,
          epochs=epoch,
          validation_data= validation_generator,
          validation_steps=num_validation_samples // batch_size
          )

Epoch 1/5
21/21 [==============================] - 57s 2s/step - loss: 4.3754 - accuracy: 0.8816 - val_loss: 1.9656 - val_accuracy: 0.9167
Epoch 2/5
21/21 [==============================] - 48s 2s/step - loss: 0.6995 - accuracy: 0.9720 - val_loss: 1.9578 - val_accuracy: 0.9375
Epoch 3/5
21/21 [==============================] - 51s 2s/step - loss: 0.3696 - accuracy: 0.9844 - val_loss: 1.4357 - val_accuracy: 0.9583
Epoch 4/5
21/21 [==============================] - 55s 3s/step - loss: 0.0356 - accuracy: 0.9907 - val_loss: 0.8514 - val_accuracy: 0.9583
Epoch 5/5
21/21 [==============================] - 46s 2s/step - loss: 0.0421 - accuracy: 0.9969 - val_loss: 3.7938 - val_accuracy: 0.9375


In [25]:
model.save('dog_cat_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [30]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import os

In [31]:
model=load_model('dog_cat_model.h5')

In [44]:
image_path='/content/train/Cat/1.jpg'
test_image=image.load_img(image_path, target_size=(224,224))
test_image=image.img_to_array(test_image)
test_image=np.expand_dims(test_image, axis=0)
# print(test_image)

In [51]:
result=np.argmax(model.predict(test_image), axis=1)

1/1 [==============================] - 0s 387ms/step


In [53]:
if result[0]==0:
  print("Cat")
else:
  print("Dog")

Cat
